# Описание
Здесь представлен baseline для [соревнования](https://stepik.org/lesson/226979/step/1?unit=199528), 
который дается в конце курса ["Введение в Data Science и машинное обучение"](https://stepik.org/course/4852/) на stepic.

За основу взять признаки которые создавали в течении курса. ROC_AUC = 0.8792.

# Импорт

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm, tqdm_notebook
import datetime

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.externals import joblib

import libs.data_helpers as dh
import libs.data_iter1 as di
import libs.utils.model_utils as mu
import libs.submit_report as rep

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Загрузка и подготовка данных

In [2]:
# загрузка данных
events  = pd.read_csv("./data/event_data_train.zip")
submissions = pd.read_csv("./data/submissions_data_train.zip")

# разделение данных для обучения на train и test
_ = dh.split_events_submissions(events, submissions, test_size=0.3)
events_train_orig, events_test_orig, submissions_train_orig, submissions_test_orig = _

# подготовка данных
X_train, y_train = di.get_x_y(events_train_orig, submissions_train_orig)
X_test, y_test = di.get_x_y(events_test_orig, submissions_test_orig)

/home/jupyter/projects/stepic_course_4852_contest/libs/data_helpers.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['date'] = pd.to_datetime(data.timestamp, unit='s')
/home/jupyter/projects/stepic_course_4852_contest/libs/data_helpers.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['day'] = data.date.dt.date


# Обучение и проверка модели

## RandomForest

In [3]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=2, 
                            min_samples_leaf=10, min_samples_split=10, 
                            class_weight='balanced')

rf.fit(X_train, y_train)
pred_proba = rf.predict_proba(X_test)
roc_score = roc_auc_score(y_test, pred_proba[:, 1])
print('roc на test', roc_score)
# должны получить roc 0.92  +- 0.02

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=10,
            min_samples_split=10, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=2, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

roc на test 0.9245751187837892


In [4]:
# важность фич
fimp = mu.get_feature_importances_df(rf.feature_importances_, X_train.columns)
fimp.head(15)

,weight
correct,0.352792
passed,0.222040
discovered,0.129428
viewed,0.128497
wrong,0.088141
started_attempt,0.067737
day,0.011365


## кроссвалидация

In [5]:
# значение к метрике на кроссвалидации коррелирует к метрике на степике

rfcv = RandomForestClassifier(**rf.get_params())

cv_scores = cross_val_score(rfcv, X_train, y_train, scoring='roc_auc', cv=10, n_jobs=-1)
mean_cv_scores = np.mean(cv_scores)
print ('mean score', mean_cv_scores)

mean score 0.9146796021598188


## Сохранение модели

In [6]:
# Расскоментируйте, если нужно сохранить модель
# !mkdir ./bin_models
# joblib.dump(rf, './bin_models/baseline_random_forest.bin')

# код для загрузки модели
# rf = joblib.load('./bin_models/baseline_random_forest.bin')

# Сделать предсказание по сабмиту

In [7]:
SUBMIT_NUM = 1

events_pred  = pd.read_csv("./data/events_data_test.zip")
submissions_pred = pd.read_csv("./data/submission_data_test.zip")
X_pred , _ = di.get_x_y(events_pred, submissions_pred)

pred_proba = rf.predict_proba(X_pred)[:, 1]
rep_df = rep.create_report(X_pred.index, pred_proba)
assert rep_df.user_id.nunique() == X_pred.index.nunique()
print ('Прогноз сохранен в файл ', rep.save_report(rep_df, SUBMIT_NUM))

print ('Распределение "вероятностей" модели')
pd.cut(pred_proba, 10).value_counts()

Прогноз сохранен в файл  ./reports/predict_2019-05-17_submit_1.csv
Распределение "вероятностей" модели


(-0.001, 0.1]    3346
(0.1, 0.2]        113
(0.2, 0.3]        169
(0.3, 0.4]        358
(0.4, 0.5]        439
(0.5, 0.6]        443
(0.6, 0.7]        259
(0.7, 0.8]        261
(0.8, 0.9]        233
(0.9, 1.0]        563
dtype: int64